In order to remove noise, to guifr the algorithm to the spot of the lesion I cropped the files to 64 on 64 on 64 bocx around the center of the lesions. The reason for the size is, from reviewing the sizes of all lesions, i found that the biggest has diameter of about 50. most are much smaller than 50. in addition 64, works welll with deep neural network as it is the 2^9

In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import SimpleITK as sitk
from rt_utils import RTStructBuilder
from skimage.transform import resize


In [2]:
# Load the CSV file into a dataframe
metadata = pd.read_csv("data/metadata.csv")

# Remove columns that contain only one unique value
for column in metadata.columns:
    if len(metadata[column].unique()) == 1:
        metadata.drop(column, axis=1, inplace=True)

metadata.rename(columns={'Study Date': 'StudyDate', 'Series Description':'course'}, inplace=True)
cols=['Series UID', 'Study UID', 'Study Description', 'Manufacturer', 'SOP Class UID', 'SOP Class Name', 'Number of Images', 'File Size','Download Timestamp']
metadata.drop(cols, axis=1, inplace=True)

# Convert the 'course' column to a numeric type, coercing errors to NaN
metadata['course'] = pd.to_numeric(metadata['course'], errors='coerce')

# Sort the DataFrame
metadata.sort_values(['pid', 'StudyDate', 'course'], inplace=True)

# Forward and backward fill the course number within each group of 'pid' and 'StudyDate'
metadata['course'] = metadata.groupby(['pid', 'StudyDate'])['course'].transform(lambda x: x.ffill().bfill())
metadata.drop('StudyDate', axis=1, inplace=True)
metadata['Modality'] = metadata['Modality'].replace({'RTSTRUCT': 'str', 'RTDOSE': 'dos', 'MR': 'mri'})
metadata.columns = ['pid','course','modality','folder']
# Convert the 'course' column to integers
metadata['course'] = metadata['course'].astype(int)
metadata

,pid,course,modality,folder
88,103,1,str,./Brain-TR-GammaKnife/GK_103/04-18-2014-NA-MR ...
11,103,1,dos,./Brain-TR-GammaKnife/GK_103/04-18-2014-NA-MR ...
90,103,1,mri,./Brain-TR-GammaKnife/GK_103/04-18-2014-NA-MR ...
91,103,2,str,./Brain-TR-GammaKnife/GK_103/12-18-2014-NA-MR ...
142,103,2,dos,./Brain-TR-GammaKnife/GK_103/12-18-2014-NA-MR ...
...,...,...,...,...
215,492,1,dos,./Brain-TR-GammaKnife/GK_492/09-08-2014-NA-hea...
223,492,1,mri,./Brain-TR-GammaKnife/GK_492/09-08-2014-NA-hea...
67,492,2,str,./Brain-TR-GammaKnife/GK_492/12-11-2014-NA-hea...
2,492,2,dos,./Brain-TR-GammaKnife/GK_492/12-11-2014-NA-hea...


In [8]:
import SimpleITK as sitk
import glob


# Function to resample image to match reference
def resample_to_reference(reference_image, target_image):
    # Resample images to match reference image dimensions, spacing, and direction
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(reference_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetTransform(sitk.Transform())
    resampler.SetOutputSpacing(reference_image.GetSpacing())
    resampler.SetSize(reference_image.GetSize())
    resampler.SetOutputDirection(reference_image.GetDirection())
    resampler.SetOutputOrigin(reference_image.GetOrigin())

    # Apply resampling
    resampled_image = resampler.Execute(target_image)
    return resampled_image

grouped_df = metadata.groupby(['pid', 'course'])

output_folder = "vox_nii"

# Iterate over each group
for group_name, group_data in grouped_df:    
    pid, course = group_name
    
    # Get the 'SOP Class Name' and 'File Location' for each group
    folders_dic = dict(zip(group_data['modality'], group_data['folder']))

    dos_folder = folders_dic['dos']
     
    mri_image_path = f"{output_folder}/mri_{pid}_{course}.nii"
    rtdose_image_path = glob.glob(os.path.join(dos_folder, "*.dcm"))[0]
    resampled_rtdose_path = f"{output_folder}/dos_{pid}_{course}.nii"

    # Load your MRI image (used as reference)
    mri_image = sitk.ReadImage(mri_image_path)

    # Load your RTDOSE image
    rtdose_image = sitk.ReadImage(rtdose_image_path)

    # Resample RTDOSE image to match MRI image
    resampled_rtdose = resample_to_reference(mri_image, rtdose_image)

    # Save the resampled RTDOSE image
    sitk.WriteImage(resampled_rtdose, resampled_rtdose_path)


Check equal between rtstruct and mri

In [28]:
import os
import nibabel as nib

# Define a function to check the equivalence of the header information
def validate_equivalence(mri_nii_path, rtstruct_nii_path):
    # Load the MRI and RTSTRUCT NIfTI files
    mri_img = nib.load(mri_nii_path)
    rtstruct_img = nib.load(rtstruct_nii_path)
    
    # Compare the header information
    # if not np.array_equal(mri_img.shape, rtstruct_img.shape):
    #     return False, "Mismatch in dimensions"
    if not np.array_equal(mri_img.header.get_zooms(), rtstruct_img.header.get_zooms()):
        return False, "Mismatch in voxel sizes"
    if not np.allclose(mri_img.affine, rtstruct_img.affine):
        return False, "Mismatch in affine transformation"
    
    # Everything matches
    return True, "All parameters match"

# Directory containing the NIfTI files
output_folder = "vox_nii"

# List all the MRI and RTSTRUCT NIfTI files
mri_files = [f for f in os.listdir(output_folder) if f.startswith('mri_') and f.endswith('.nii')]
rtdose_files = [f for f in os.listdir(output_folder) if f.startswith('dos_') and f.endswith('.nii')]

# Assume a naming convention that allows matching RTSTRUCT files to MRI files
for mri_file in mri_files:
    # Extract patient ID and course from the file name
    parts = mri_file.split('.')[0].split('_')
    pid = parts[1]
    course = parts[2]
    
    # Find the matching RTSTRUCT file
    rtdose_file = next((f for f in rtdose_files if f"dos_{pid}_{course}" in f), None)
    
    if rtdose_file:
        # Validate equivalence of header information
        mri_nii_path = os.path.join(output_folder, mri_file)
        rtdose_nii_path = os.path.join(output_folder, rtdose_file)
        valid, message = validate_equivalence(mri_nii_path, rtdose_nii_path)
        if not valid:
            print(f"Validation for {mri_file} and {rtdose_file}: {message}")
    else:
        print(f"No matching RTSTRUCT file found for {mri_file}")


resize to 256x256x256

In [29]:
import os
import numpy as np
import nibabel as nib

input_dir = 'vox_nii'
output_dir = 'ppp_nii'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.startswith('dos') and filename.endswith(".nii"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        # Load the NIfTI file
        img = nib.load(input_path)
        data = img.get_fdata()

        # Resize the first two dimensions (x and y) with padding or cropping
        x_diff = 256 - data.shape[0]
        y_diff = 256 - data.shape[1]

        if x_diff > 0:  # Pad x dimension
            x_pad_before = x_diff // 2
            x_pad_after = x_diff - x_pad_before
            data = np.pad(data, ((x_pad_before, x_pad_after), (0, 0), (0, 0)), 'constant')
        elif x_diff < 0:  # Crop x dimension
            x_crop_before = abs(x_diff) // 2
            x_crop_after = abs(x_diff) - x_crop_before
            data = data[x_crop_before:-x_crop_after, :, :]

        if y_diff > 0:  # Pad y dimension
            y_pad_before = y_diff // 2
            y_pad_after = y_diff - y_pad_before
            data = np.pad(data, ((0, 0), (y_pad_before, y_pad_after), (0, 0)), 'constant')
        elif y_diff < 0:  # Crop y dimension
            y_crop_before = abs(y_diff) // 2
            y_crop_after = abs(y_diff) - y_crop_before
            data = data[:, y_crop_before:-y_crop_after, :]

        # Resize the third dimension (z) with padding
        z_diff = 256 - data.shape[2]
        if z_diff > 0:
            z_pad_before = z_diff // 2
            z_pad_after = z_diff - z_pad_before
            data = np.pad(data, ((0, 0), (0, 0), (z_pad_before, z_pad_after)), 'constant')

        # Create a new NIfTI image with the resized data
        new_img = nib.Nifti1Image(data, img.affine)

        # Save the resized image
        nib.save(new_img, output_path)
